## 1. Import Library

In [ ]:
import glob
import csv
import pandas as pd
import numpy as np

## 2. Import Data
Import annual data of 2016 and 2017 respectively.

In [ ]:
files = glob.glob('G:\\Data\\*.xls')

data = pd.DataFrame() 
for file in files:
    temp = pd.read_excel(file, encoding = 'utf-8')
    # Delete the last row which sums up the purchase of the entire month
    temp = temp.drop(temp.index[len(temp)-1])
    data = data.append(temp)

## 3. Clean Dataset

### 3.1. Drop Columns

In [ ]:
# Drop Column Named "Total Qty"
data = data.drop(['Total Qty'], 1)

In [ ]:
# Drop Columns Containing "%"
data = data[data.columns.drop(list(data.filter(regex = '%')))]

### 3.2. Reshape Dataframe

In [ ]:
# Coulmn Axis: Purchase_Date, Supplier
# Row Axis: Fruit_Type_ID
# Value: Qty
df = data.melt(id_vars=['Purchase_Date', 'Supplier'], var_name = 'Fruit_Name_ID', value_name = 'Qty')

### 3.3. Data Type Conversion

In [ ]:
# Records in Column "value" turned out to be string objects rather than numbers.
# Find out that numbers formatted with thousand separator are considered as strings. 
df.dtypes

### 3.3.1. Convertion to String

In [ ]:
df['Qty'] = df['Qty'].astype(str)

### 3.3.2. Remove Thousand Separator

In [ ]:
df['Qty'] = df['Qty'].apply(lambda x: x.replace(',',''))

### 3.3.3. Conversion to Integer

In [ ]:
df['Qty'] = df['Qty'].astype(int)

## 3.4. Export Dataset for Analysis
Export annual data of 2016 and 2017 respectively.

In [ ]:
df = df.groupby(['Purchase_Date','Supplier','Fruit_Name_ID'])['Qty'].sum().reset_index()

In [ ]:
df.to_csv('G:\\Analysis.csv', index = False, encoding = 'ansi')

## 4. Pivot Analysis

### 4.1. Import Data
Import annual data of 2016 and 2017 respectively.

In [ ]:
df = pd.read_csv('G:\\Analysis.csv', encoding = 'ansi')

### 4.2. Object Definition

In [ ]:
# Definition 1：Months
months = ['/01/','/02/','/03/','/04/','/05/','/06/','/07/','/08/','/09/','/10/','/11/','/12/']

# Definition 2.1：Suppliers in 2017 
new_column_2016 = ['Farm 1','Farm 2','Farm 3-1','Farm 4']

# Definition 2.2：Suppliers in 2018 
new_column_2017 = ['Farm 1','Farm 2','Farm 3-2','Farm 4']

# Definition 3：Fruits
new_index = ['APPL001','APPL002','Apple','GRAP001','Grape','KIWI001','Kiwifruit']

### 4.3. Pivot Analysis

In [ ]:
for month in months:
    
    # Select data of specific month. 
    df_month = df[df['Purchase_Date'].str.contains(month)]
    
    # Drop Column "Purchase_Date."
    df_month = df_month.drop(['Purchase_Date'], 1)
    
    # Conduct Pivot Analysis.
    df_pivot = df_month.pivot_table(index='Fruit_Name_ID', columns='Supplier', values='Qty', aggfunc=np.sum)
      
    # Determine which year it is by suppliers.
    # Farm 3-1 is the supplier of 2016.
    if 'Farm 3-1' in list(df_pivot.columns.values):
        new_column = new_column_2016
        year = '2016'
    # Farm 3-2 is the supplier of 2017.
    elif 'Farm 3-2' in list(df_pivot.columns.values):
        new_column = new_column_2017
        year = '2017'
    # Suppose that Farm 3-1 and Farm 3-2 are not regular suppliers and it is likely that 
    # there is no orders distributed to these two during some months of the year accordingly, 
    # determining which year it is should be referring to "Purchase_Date" instead.
    else:
        # Since it is the annual data of either 2016 or 2017 to be imported, 
        # all records listed should be sharing the same "Purchase_Year" and 
        # selecting the first record for inspection will be sufficient.        
        for inspection will be enough.
        if '2016' in df['Purchase_Date'][0]:
            new_column = new_column_2017
            year = '2016'
        elif '2017' in df['Purchase_Date'][0]:
            new_column = new_column_2018
            year = '2017'            
        
    # Rename columns 
    # 2016: Farm 1, 2, 3-1, 4
    # 2017: Farm 1, 2, 3-2, 4
    df_column_fixed = df_pivot.reindex(new_column, axis = 'columns')
    
    # Rename indexes
    df_row_fixed = df_column_fixed.reindex(new_index)
    
    # Rearrange indexes by replacing "Fruit_Name_ID" with "Fruit_Type." 
    # Apple = APPL001 + APPL002
    df_row_fixed.loc['Apple'] = df_row_fixed.loc['APPL001'] + df_row_fixed.loc['APPL002']
    # Grape = GRAP001
    df_row_fixed.loc['Grape'] = df_row_fixed.loc['GRAP001']
    # Kiwifruit = KIWI001
    df_row_fixed.loc['Kiwifruit'] = df_row_fixed.loc['KIWI001']
    # Drop indexes named according to the list of "Fruit_Name_ID."
    df_row_fixed = df_row_fixed.drop(index=['APPL001','APPL002','GRAP001','KIWI001'])
    
    # Replace NA value with zero.
    df_fillna = df_row_fixed.fillna(0)
    
    # Retain indexes when exporting data 
    df_done = df_fillna.reset_index()
    
    # Export monthly data
    df_done.to_csv("G:\\Analysis_" + year + "_" + month.replace('/','') + "_Result.csv", index = False, encoding = 'ansi')